# Tutorial de clasificación de imágenes

## Introducción

En este tutorial aprenderemos a utilizar modelos de redes neuronales convolucionales preentrenados para clasificación de imágenes. La clasificación de imágenes es una de las subtareas más conocidas en visión por computador. Esta se basa enteramente en la tarea del reconocimiento, pues consiste en asignar una categoría semántica a cada imagen según el objeto principal que se encuentra en ella. Por este motivo, es necesario utilizar imágenes que muestren un único objeto o que muestren varios objetos de una misma categoría. Cabe aclarar, que también es posible utilizar imagenes con múltiples objetos de diferentes categorías y luego predecir todas las categorías presentes en la imagen. Sin embargo, este último es una versión más complicada del problema de clasificación que no estudiaremos en este tutorial.

En la actualidad, existen varias bases de datos que permiten entrenar y evaluar algoritmos en esta tarea. No obstante, la principal es la base de datos de ImageNet. Esta base de datos contiene más de 14 millones de imágenes de objetos pertenecientes a más de 21 mil categorías semánticas. Aunque ImageNet fue publicada inicialmente en el 2009, esta se popularizó en el 2010 debido a que fue utilizada para crear el "challenge de reconocimiento visual alargo escala en ImageNet" o IMLSVRC por sus siglas en inglés. Este challenge es de gran importancia en la historia de la inteligencia artificial, pues en el 2012 Alex Krizhevsky, Illa Suteskever y Geoffry Hinton ganaron este challenge (y con una gran diferencia) utilizando por primera vez una red neuronal convolucional. La red creada por ellos, mejor conocida como AlexNet, demostró el poder del aprendizaje profundo (deep learning) y de las redes neuronales convolucionales, marcando así una nueva era para inteligencia artificial. En este tutorial, utilizaremos tanto AlexNet como otras arquitecturas de redes neuronales convolucionales para clasificar 20 imágenes aleatorias tomadas de internet. Todos y cada uno de estos modelos fueron entrenados en la versión de ImageNet que se utiliza para IMLSVRC, el cual contiene únicamente 1000 clases. Posteriormente, utilizaremos las métricas oficiales del challenge que son el accuracy (o exactitud) y el top5 accuracy (que permite al modelo dar 5 posibles clasificaciones). 

### Paso 1: Importación de librerías

Para comenzar debemos importar las librerías necesarias. A continuación, podrán encontrar las librerías que usaremos para el tutorial. Primero, necesitaremos numpy para el manejo de matrices y vectores. Segundo, usaremos os (operating system) para el manejo correcto de rutas y archivos. Tercero, necesitaremos json para poder cargar archivos tipo json. Y finalmente, utilizaremos OpenCV como la librería principal para los algoritmos de clasificación. OpenCV es una librería muy conocida para visión por computador en C++ y python. Esta contiene múltiples métodos, algoritmos, funciones y modelos para múltiples tareas de procesamiento de imagenes. En este caso, la usaremos para obtener las redes neuronales convolucionales preentrenadas para clasificar imágenes. Esta librería no viene importada en las librerías por defecto de python. Por ende, tendremos que instalarla con el siguiente comando.

`pip install opencv-python`

Y la librería es importada desde python con la línea: import cv2

In [10]:
import numpy as np 
import cv2 # librería OpenCV
import os
import json

### Paso 2: Función para cargar las clases de ImageNet
La siguiente función se encarga de leer el archivo de texto (.txt) que contiene las clases de ImageNet, este archivo se llama "clases_ImageNet.txt". En este tutorial nos basaremos en la version de ImageNet que se utiliza para IMLSVRC, la cual contiene unicamente 1000. Por este motivo, este archivo contiene el nombre completo de cada una de las clases de las 1000 clases como lineas del archivo. Es recomendable que usted abrá el archivo y lo revise. Allí notará que el nombre de algunas categorías son varios sinonimos separados por comas, en esos casos el nombre principal es el que aparece de primero. 

In [11]:
def cargarClasesImageNet():
    with open('clases_ImageNet.txt','r') as f:
        clases = f.readlines()
        clases = [c.strip() for c in clases]
    return clases

### Paso 3: Función para cargar una red preentrenada para clasificación.

En este paso usted creará la función para obtener una red neuronal convolucional preentrenada en clasificación de imágenes en ImageNet. Como se mencionó previamente, para ello utilizaremos la librería OpenCV que se importa como cv2. Esta librería tiene al interior el paquete dnn que significa "deep neural networks". Este paquete ofrece una gran variedad de funciones, métodos y modelos para el uso y manipulación de redes neuronales profundas. En especial, tiene varias funciones que nos permitirán crear un diseño de red neuronal y también cargar los valores de todos sus parámetros (es decir, cargar todos los pesos de la red). En este caso, utilizaremos la función `readNetFromCaffe(<prototxt>,<caffemodel>)`. Esta función se encarga de crear una red neuronal según un archivo de texto que contiene las especificaciones de diseño de dicha red (número de capas, el tipo de cada capa, tamaño de la capa, etc.), y luego asignar los pesos correspondientes de cada capa según un archivo con el formato Caffe. Caffe es una librería de aprendizaje profundo (deep learning) creada por la Universidad de California, Berkley para C++. Esta fue una de las primeras librerías en la historia para creación y manipulación de redes profundas y es en la cual muchas de las primeras redes neuronales convolucionales fueron creadas. Esta librería, permite almacenar los pesos de una red previamente entrenada en un archivo con un formato específico codificado por la misma librería. Así, es posible serializar las redes para ser cargadas o reutilizadas después. En este tutorial, utilizaremos este formato para cargar los pesos de nuestras redes neuronales convolucionales de clasificación y esto nos los permitirá la función `readNetFromCaffe(<prototxt>,<caffemodel>)` de OpenCV. Finalmente, esta función retornará un objeto de tipo de cv2.dnn.Net que es la red neuronal cargada. La documentación oficial de la función se encuentra en el siguiente [enlace](https://docs.opencv.org/3.4/d6/d0f/group__dnn.html#ga29d0ea5e52b1d1a6c2681e3f7d68473a).
    
Ahora bien, esta función tiene dos parámetros obligatorios que son `<prototxt>` y `<caffemodel>`. El primer parámetro, (`<prototxt>`) es un archivo de texto con extensión .prototxt que contiene el diseño de la red. El segundo parámetro (`<caffemodel>`) es un archivo de extensión .caffemodel que contiene los pesos preentrenados de la red. Para mayor facilidad, usted contará con el archivo .prototxt y el archivo .caffemodel para todas las redes que probaremos en este tutorial. Estos se encuentran al interior del directorio llamado "modelos". En este directorio usted encontrará 4 subdirectorios, cada uno titulado con el nombre de un modelo de red neuronal convolucional famosa para la clasificación de imágenes en ImageNet. Al interior de cada uno de estos directorios usted encontrará un archivo llamado *modelo.prototxt* que contiene el diseño de dicha red y otro archivo llamado *pesos.caffemodel* que contiene los pesos en formato Caffe. Le recomendamos dar un vistazo rápido a los archivos .prototxt en el bloc de notas o cualquier otro editor de texto. Allí podrá observar como las redes es componen de varias capas convolucionales y algunas capas lineales al final para realizar la clasificación. Por el contrario, tenga en cuenta que los archivos .caffemodel se encuentran codificados y no pueden ser leídos en un editor de texto. Adicionalmente, encontrará un archivo llamado *informacion.txt* que contiene alguna información general de cada modelo que le podrá ser útil para sacar conclusiones al respecto de sus resultados. Finalmente, su función en este paso es crear una función que reciba el nombre de un modelo por parámetro (es decir, el nombre de los un subdirectorio al interior del directorio *modelos*) y retorne el objeto tipo cv2.dnn.Net que contiene el modelo ya creado y cargado con los pesos correspondientes. Tenga en cuenta, que la función retornará error si se ingresa una ruta incorrecta o si se ingresan pesos que no corresponden al diseño de red especificado.

In [3]:
def crearClasificador(nombre_modelo: str):

    ruta_prototxt = os.path.join('modelos', "GoogleNet", 'modelo.prototxt')

    ruta_caffemodel = os.path.join('modelos', "DenseNet121", 'pesos.caffemodel')
    
    modelo = cv2.dnn.readNetFromCaffe(ruta_prototxt, ruta_caffemodel)
    
    return modelo

### Paso 4: Función para crear imágenes aptas para los modelos

En este paso crearemos una función que cargue una serie de imágenes y les hagas las transformaciones necesarias para poder clasificarlas con una red neuronal convolucional. En otras palabras, este paso consiste en realizar el preprocesamiento necesario en las imágenes para hacerlas entradas aptas a una red neuronal convolucional. Este preprocesamiento consiste en dos principales modificaciones a las imágenes. La primera modificación es una reducción de tamaño, pues las imágenes naturales suelen tener tamaños muy grandes que hacen que el procesamiento de la red sea mucho más pesado y lento. Por ende, para hacer que las redes puedan clasificar las imágenes más rápido y que sean menos pesadas, el tamaño de estas debe ser reducido a un valor pequeño que permita procesar la imagen de forma ligera pero que igual puedan distinguirse los objetos que contiene. La segunda transformación es una estandarización de los valores de los pixeles de las imágenes. Esta última se realiza restándole a cada canal cromático un valor predeterminado y luego dividiendo la imagen por un factor escalador. Por lo general, se utilizan las medias de cada canal cromático en todas las imágenes de la base de datos de entrenamiento como valor para la resta, y se utiliza la desviación estándar de las imágenes como factor escalador. 

Dada esta información crearemos una función que haga todo este preprocesamiento. Para ello, nos apoyaremos en la función `blobFromImages(<imagenes>,<factor>,<tamano>,<media>)` del módulo dnn de cv2. Esta función se encarga de realizar todos los pasos de preprocesamiento previamente explicados. Así, esta función tiene 4 parámetros. El primero es `<imagenes>` y es una lista con las imágenes originales (que se cargan como matrices de numpy) en formato BGR. El segundo es `<factor>` y es el factor escalador por el cual se dividirá la imagen. El tercero es `<tamano>` y es una tupla de enteros de longitud 2 con el tamaño al cual se desea reducir las imágenes (una tupla que indica `(número de filas , número de columnas)`). Y el cuarto es `<medias>` y es una tupla de floats de longitud 3 con los valores de las medias para restar a cada canal cromático en la estandarización. Finalmente, esta función retornará una matriz de numpy de dimensiones `(numero de imagenes, numero de canales , número de columnas, número de filas)` que representa el arreglo con todas las imágenes reducidas y estandarizadas juntas. El orden del arreglo es igual al orden de la lista de imágenes especificada en el parámetro `<imagenes>`. La documentación oficial de esta función se encuentra en este [enlace](https://docs.opencv.org/3.4/d6/d0f/group__dnn.html#ga0b7b7c3c530b747ef738178835e1e70f). 

Para nuestra función, utilizaremos como entrada una lista con los nombres de las imágenes a ser cargadas, así como el tamaño al cual reducir la imagen y los valores para hacer la estandarización. Finalmente, retornaremos tanto la matriz de numpy con las imágenes después de las transformaciones realizadas, como la lista de matrices de numpy con las imágenes originales para poder hacer visualizaciones si se desea. Así mismo, para cargar las imágenes utilizaremos la función `imread(<ruta_imagen>)` de OpenCV que lee un archivo de imagen en la ruta dada por parámetro y retorna el arreglo de numpy que contiene la imagen en BGR. Tenga en cuenta que la función recibe la ruta hacia el archivo de la imagen con la extensión del mismo incluido, es decir que no funcionará con solo indicarle el nombre de la imagen. Por ende, se debe indicar una cadena del estilo `"imagenes/nombre_imagen.jpeg"` que indique el directorio y el nombre del archivo a leer. Por último, considere que tanto la lista con las imágenes cargadas, como el arreglo con las imágenes procesadas debe tener el mismo orden que la lista de nombres que entra por parámetro.

In [ ]:
def cargarImagenesClasificacion(nombres_imagenes: list, dst: float, tamano: tuple, medias: tuple):
    
    imagenes = []
    blobs = []
    
    for nombre in nombres_imagenes:
        if not nombre.endswith('.jpeg'):
            nombre += '.jpeg'
        
        ruta_imagen = os.path.join('clases_ImageNet.txt', nombre)
        imagen = cv2.imread(ruta_imagen)
        
        if imagen is not None:
            imagenes.append(imagen)
            
            blob = cv2.dnn.blobFromImage(imagen, scalefactor=1.0/dst, size=tamano, mean=medias)
            blobs.append(blob)
    
    entrada = np.concatenate(blobs, axis=0) if blobs else np.array([])
    
    return imagenes, entrada


### Paso 5: Función para clasificar un conjunto de imágenes

En este paso crearemos la función principal para clasificar un conjunto de imágenes. Este es sin duda el paso más importante. Para hacer la clasificación de las imágenes primero nos apoyaremos en las funciones que creamos previamente. Es decir, utilizaremos la función crearClasificador para obtener nuestro modelo de clasificación, y la función cargarImagenesClasificación para cargar y procesar una lista de imágenes para ser clasificadas. Note que los parámetros de la función que crearemos en este paso son similares a los parámetros de las funciones creadas anteriormente con el fin de poder llamar a las otras funciones desde esta que crearemos. Posteriormente, utilizaremos algunos métodos propios de la clase cv2.dnn.Net para hacer la clasificación, recuerde que el modelo de clasificación que cargamos es una instancia de esta clase. Específicamente, usaremos primero el método `setInput(<entrada>)` que recibe el parámetro `<entrada>`, el cual es una matriz de numpy con las imágenes que usaremos como entrada. Este método se encarga de asignar la matriz pasada por parámetro como la entrada que será procesada por la red. La documentación oficial se encuentra [acá](https://docs.opencv.org/4.x/db/d30/classcv_1_1dnn_1_1Net.html#a5e74adacffd6aa53d56046581de7fcbd). De segundo, utilizaremos la función `forward()` que indica al modelo que debe procesar la entrada y calcular las predicciones. Este método no recibe ningún parámetro y retorna una matriz de numpy con la salida calculada por la red neuronal. La documentación oficial la podrá encontrar [acá](https://docs.opencv.org/4.x/db/d30/classcv_1_1dnn_1_1Net.html#a98ed94cb6ef7063d3697259566da310b). Tenga en cuenta que estos métodos son propios de la clase cv2.dnn.Net. Lo que implica que para utilizarlos primero debe haber creado el modelo (cargado en un variable llamada modelo_creado, por ejemplo) y luego llamar a los métodos mediante `modelo_creado.setInput(<entrada>)` y `modelo_creado.forward()`.

Una vez obtenida la salida de la red podemos obtener la clasificación exacta de la red. Para ello, considere dos cosas. Primero, que la salida de la red tendrá dimensiones `(numero de imagenes , 1000)`. Lo cual se debe a que las redes neuronales están diseñadas para poder procesar y predecir sobre varias entradas a la vez, por ende, la primera dimensión corresponde al número de imágenes en la entrada. Así, si procesáramos 10 imágenes, la primera dimensión tiene tamaño 10. Por supuesto, la segunda dimensión tiene tamaño 1000 debido a que este es el número de clases posibles en ImageNet para la clasificación. Como es de esperase, el orden de las predicciones de salida de la red es igual al orden de las imágenes en la entrada. El segundo aspecto para considerar es que la salida de la red contiene los valores resultantes después de los múltiples cálculos realizados por la red, y por ende estos valores se encuentran en el rango de todos los números reales. Por ello, es necesario aplicar una transformación adicional para convertir este vector en una distribución de probabilidad sobre las 1000 clases que indique cuál es la probabilidad de que cada imagen en la entrada pertenezca a cada clase. Esta operación se puede realizar utilizando una función matemática muy famosa conocida como el softmax. El softmax es una transformación que se encarga de convertir cada valor de un vector que se encuentra en el rango de los numero reales al rango de 0 a 1, pero asegurando que la suma total del vector sea aproximadamente 1. Para más información, en internet encontrará bastante contenido acerca de la función softmax. Dicho esto, el softmax nos permitirá convertir la salida de la red en distribuciones de probabilidad por imagen. La ecuación del softmax es la siguiente:

$Softmax(\vec{y}) = \frac{e^{\vec{y}}}{\sum{e^{\vec{y}}}}$

Tenga en cuenta que se requiere aplicar el softmax a cada predicción por imagen en la entrada, es decir si la entrada tiene 10 imágenes la salida tendrá dimensiones `(10 , 1000)` y se debe aplicar el softmax a cada una de las 10 filas de esta matriz independientemente. Una vez aplicado el softmax a la salida de nuestra red tendremos nuevamente una matriz de dimensiones `(numero_de_imagenes , 1000)` que en cada posición $(i,j)$ contiene la probabilidad de que la imagen de entrada $i$ pertenezca a la clase $j$. Estas probabilidades son también conocidas como confianzas, es decir que son el valor de seguridad que tiene el modelo de que la cada imagen de entrada pertenezca a cada clase. La clase final en la cual es clasificada cada imagen es la clase que tenga la mayor confianza (mayor probabilidad). Así mismo, se podría tomar más de una sola clase y más bien tomar 2, 3 o 5 clases (las que tengan las mayores confianzas) para así permitir al algoritmo tener espacio de error y poder hacer varias clasificaciones por imagen. En este caso, la función que crearemos toma por parámetro la lista con los nombres de todas las clases en el orden correcto, para así saber el nombre de las clases predichas por el algoritmo según sus índices. En esta función retornaremos la lista con imágenes originales, los índices de las clases principales predichas (las de mayor probabilidad o confianza) por imagen, los nombres de las clases principales predichas, las confianzas de las clases principales predichas, y una lista de vectores con las 5 clases de mayor confianza. Nuevamente, el orden de todas las variables de salida debe coincidir con el orden de la lista de nombres de imágenes que se recibe por parámetro.

In [5]:
def clasificar(nombres_imagenes: list, clases_imagenet: list, modelo: str, dst: float, tamano: tuple, medias: tuple):

    modelo_cnn = crearClasificador(modelo)

    imagenes, entrada = cargarImagenesClasificacion(nombres_imagenes, dst, tamano, medias)

    modelo_cnn.setInput(entrada)

    salida_red = modelo_cnn.forward()

    probabilidades = cv2.dnn.softmax(salida_red, axis=1)

    indices = []
    clases = []
    confianzas = []
    top5 = []

    for probabilidad in probabilidades:
        top5_indices = np.argsort(probabilidad)[::-1][:5]
        top5.append(top5_indices.tolist())

        indice_principal = top5_indices[0]
        indices.append(indice_principal)

        clase_principal = clases_imagenet[indice_principal]
        clases.append(clase_principal)

        confianza_principal = probabilidad[indice_principal]
        confianzas.append(confianza_principal)

    return imagenes, indices, clases, confianzas, top5


### Paso 6: Funciones para calcular métricas

En este paso calcularemos las métricas oficiales del challenge de clasificación de imágenes en ImageNet. Como se mencionó previamente estas métricas son el accuracy y el top-5-accuracy. El accuracy, o la exactitud en español, es una medida el porcentaje de predicciones correctas. Es decir, este mide la cantidad de predicciones acertadas entre todas las predicciones realizadas. Así, por ejemplo, si queremos clasificar 20 imagenes y solo 9 de ellas quedan bien clasificadas, entonces el accuracy será de 45%. De manera matemática, el accuracy se define como:

$Accuracy = \frac{Predicciones\_correctas}{Total\_de\_predicciones}$

Dado esto, el challenge de ImageNet considera tanto el accuracy estricto como el top-5-accuracy. El accuracy estricto, o solo accuracy, considera una predicción correcta a aquellas en las que la clase de mayor confianza predicha por el modelo corresponde a la misma clase de la anotación de la imagen (por ejemplo, clasificar como perro una imagen cuya anotación es "perro"). Por el contrario, el top-5-accuracy considera una predicción correcta a aquellas donde una de las 5 clases con mayor confianza predichas por el modelo corresponde a la misma clase de la anotación de la imagen (por ejemplo, que las 5 clases predichas de mayor confianza sean perro, lobo, sabueso, zorro y coyote, para una imagen cuya anotación es zorro). Ahora, con esto en mente, crearemos dos funciones: una para calcular el accuracy estricto y otra para el top-5-accuracy. Ambas funciones reciben dos parámetros, uno llamado `<anotaciones>` que es una lista con los indices de las clases anotadas para cada imagen, y otro llamado `<predicciones>` que es una lista con los indices de las clases predichas para cada imagen. Por supuesto, para la función del accuracy estricto la lista de predicciones debe tener en cada posición un unico numero por imagen, y para la función del top-5-accuracy la lista debe tener en cada posición una lista con los 5 indices predichos. Finalmente, tenga en cuenta el accuracy y el top-5-accuracy debe ser un número entre 0 y 1.

In [7]:
def calcularAccuracy(anotaciones:list, predicciones:list):
    
    total_predicciones = len(anotaciones)

    correctas = sum([anotaciones[i] == predicciones[i] for i in range(total_predicciones)])

    accuracy = correctas / total_predicciones
    
    return accuracy

def calcularTop5Accuracy(anotaciones:list, predicciones:list):

    total_predicciones = len(anotaciones)
    
    correctas = sum([anotaciones[i] in predicciones[i] for i in range(total_predicciones)])

    top5_accuracy = correctas / total_predicciones

    return top5_accuracy

### Paso 7: Clasificar las imagenes con cada modelo y calcular las métricas.

Para finalizar, utilizaremos todas las funciones creadas previamente para clasificar las 20 imágenes aleatorias. Las imágenes se encuentran al interior del directorio llamado "imagenes". Como se mencionó previemente, todas son imágenes de internet en formato .jpg de objetos aleatorios. Así mismo, las información de los modelos la encontrará en el directorio llamado "modelos" que contiene 4 subdirectorios con el nombre de 4 arquitecturas de redes neuronales convolucionales famosas. Estas son: AlexNet, DenseNet 121, GoogleNet y VGG 19. Al interior de de cada uno de estos subdirectorios encontrará un archivo llamado modelo.prototxt con el diseño del modelo, otro llamado pesos.caffemodel con los valores d elos parámetros del modelo, y un archivo llamado informacion.txt con información adicional del modelo. Por otro lado, hay tres archivos que usted tendrá que utilizar. El primero, es el archivo llamado clases_ImageNet.txt con el nombre de todas las clases del challenge de ImageNet. No obstante, la función cargarClasesImageNet() se encarga de cargar y leer este archivo. Segundo, el archivo anotaciones.json es un archivo de tipo json que contiene las las anotaciones de clases para cada imagen. Al archivo contiene un diccionario cuyas llaves son los nombres de las 20 imágenes (sin el .jpeg), y los valores son diccionarios con dos llaves. La primera es "indice_clase" y su valor es el indice de la clase de imagenet a la que pertenece la imagen, La segunda es "nombre_clase" y su valor es el nombre completo de la clase a la que pertenece. De manera resumida, el archivo llamado "anotaciones.json" tiene la siguiente estuctura:

`{<nombre de la imagen1>: {"indice_clase": (indice de la imagen), "nombre_clase": (nombre de la clase)}, <nombre de la imagen2>: {"indice_clase": (indice de la imagen), "nombre_clase": (nombre de la clase)}, ...}`

Le recomendamos que abra el archivo en algún editor de texto y lo revise. El tercer archivo a considerar es el archivo llamado "hiperparametros.json". Al igual que el anterior, este es un archivo json que contiene un diccionario en su interior. Las llaves del diccionario son los nombres de las arquitecturas al interior de las carpeta llamada "modelos". Los valores del diccionario son un diccionario con tres llaves, la primera es "desviacion" y su valor es el valor de desviación estandar que se debe utilizar como factor escalador al estandarizar las imágenes de entrada para cada modelo. La segunda es "medias" y su valor es una lista de longitud 3 con los valores de las medias para restar a cada canal durante la estadarización. Y la tercera es "tamano" y es una lista de longitud 2 con el tamaño (numero de filas y columnas) al que se debe reducir la imagen durante la estandarización. Igualmente, se recomienda abrir este archivo en un editor de texto y revisarlo.

Ahora bien, es momento de clasificar las imágenes ! Su tarea es, con cada uno de los 4 modelos, utilizar las funciones creadas previamente para clasificar las 20 imagenes del directorio. Para cada modelo usted deberá clasificarl las 20 imágenes y calcular tanto el accuracy como el top-5-accuracy. Para poder hacer su resultado evaluable es importante que usted cree una variable llamada `resultados` que contenga un diccionario cuyas llaves sean los nombres de los modelos y cuyos valores sean un tupla de longitud dos con el accuracy de primero y el top-5-accuracy de segundo para cada modelo. Es decir el diccionario debe tener la siguiente estructura:

`resultados = {<nombre del modelo1>: (accuracy,top-5-accuracy), <nombre del modelo2>: (accuracy,top-5-accuracy), ...}`

Recuerde nombrar bien la variable o de lo contrario su calificación disminuirá. Finalmente, lo invitamos a que visualize los resultados. Es decir, observe las imagenes detalladamente e imprima la o las clases predicchas para dicha imagen con cada modelo. Así usted podrá tener una mejor idea del desempeño y las capacidades de cada modelo.

In [ ]:
clases_imagenet = cargarClasesImageNet()

with open('anotaciones.json', 'r') as file:
    anotaciones = json.load(file)

indices_anotaciones = [anotaciones[imagen.split('.')[0]]['indice_clase'] for imagen in os.listdir('imagenes') if imagen.endswith('.jpg')]

with open('hiperparametros.json', 'r') as file:
    hiperparametros = json.load(file)

modelos = ["AlexNet", "DenseNet121", "GoogleNet", "VGG19"]

nombres_imagenes = [f for f in os.listdir('imagenes') if f.endswith('.jpg')]

resultados = {}

for modelo in modelos:
    modelo_cnn = crearClasificador(modelo)
    dst = hiperparametros[modelo]['desviacion']
    tamano = tuple(hiperparametros[modelo]['tamano'])
    medias = tuple(hiperparametros[modelo]['medias'])

    _, predicciones, _, _, top5_predicciones = clasificar(
        nombres_imagenes, clases_imagenet, modelo_cnn, dst, tamano, medias)

    accuracy = calcularAccuracy(indices_anotaciones, predicciones)
    top5_accuracy = calcularTop5Accuracy(indices_anotaciones, top5_predicciones)

    resultados[modelo] = (accuracy, top5_accuracy)

print(resultados)